## FABlib API References Examples

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete) 

In [1]:
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jakejongejans_0000313927
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [2]:
slice_name = 'DYNAMOS-on-FABRIC'
image = "default_ubuntu_24"

# Host of the site used for the nodes (Use specific host to avoid error occurred without it: "Timeout waiting for the server to come up")
site = "AMST"
site_host = "amst-w1.fabric-testbed.net"

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
node3_nic_name = 'NIC3'
network_name = 'NET1'

In [3]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add Nodes with the specific variables
# Also validate the node can be created and raise an exception in case of failure
node1 = slice.add_node(name=node1_name, site=site, disk=20, validate=True, raise_exception=True, host=site_host, image=image)
node2 = slice.add_node(name=node2_name, site=site, disk=20, validate=True, raise_exception=True, host=site_host, image=image)
node3 = slice.add_node(name=node3_name, site=site, disk=20, validate=True, raise_exception=True, host=site_host, image=image)

iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
iface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]

net1 = slice.add_l3network(name=network_name, interfaces=[iface1, iface2, iface3], type='IPv4')

# Calculate the lease end time for 2 weeks from now with timezone information
lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

# Submit the slice, using an end date 2 weeks from now (the current maximum lease time) 
# to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
# Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time);


Retry: 10, Time: 262 sec


ID,3bfcd891-35b2-472c-97d5-53890999ab25
Name,DYNAMOS-on-FABRIC2
Lease Expiration (UTC),2025-04-25 11:42:47 +0000
Lease Start (UTC),2025-04-11 11:42:48 +0000
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
f66e4b64-de5c-4973-bc5c-082ac3842d10,Node1,2,8,100,default_ubuntu_24,qcow2,amst-w1.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe1d:e569,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe1d:e569,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
9e36cef7-d5b9-43ee-ac1f-af90698f6ab9,Node2,2,8,100,default_ubuntu_24,qcow2,amst-w1.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fea7:716,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fea7:716,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0ec89cf2-b6df-4679-b168-7f7a8f813fc8,Node3,2,8,100,default_ubuntu_24,qcow2,amst-w1.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe2b:19a2,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe2b:19a2,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
5c3ad424-cdfa-410e-b920-19528a73ba9a,NET1,L3,FABNetv4,AMST,10.145.5.0/24,10.145.5.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-NIC1-p1,p1,Node1,NET1,100,config,,16:92:0A:AB:66:74,enp7s0,enp7s0,fe80::1492:aff:feab:6674,6,HundredGigE0/0/0/5
Node2-NIC2-p1,p1,Node2,NET1,100,config,,16:69:4F:CC:FB:1C,enp7s0,enp7s0,fe80::1469:4fff:fecc:fb1c,6,HundredGigE0/0/0/5
Node3-NIC3-p1,p1,Node3,NET1,100,config,,16:F1:B5:FA:DD:E6,enp7s0,enp7s0,fe80::14f1:b5ff:fefa:dde6,6,HundredGigE0/0/0/5



Time to print interfaces 272 seconds


In [4]:
slice.list_nodes();

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
f66e4b64-de5c-4973-bc5c-082ac3842d10,Node1,2,8,100,default_ubuntu_24,qcow2,amst-w1.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe1d:e569,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe1d:e569,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
9e36cef7-d5b9-43ee-ac1f-af90698f6ab9,Node2,2,8,100,default_ubuntu_24,qcow2,amst-w1.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fea7:716,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fea7:716,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0ec89cf2-b6df-4679-b168-7f7a8f813fc8,Node3,2,8,100,default_ubuntu_24,qcow2,amst-w1.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe2b:19a2,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe2b:19a2,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


In [5]:
#node = slice.get_node('Node1')

for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')

Hello, FABRIC from node Node1
Hello, FABRIC from node Node2
Hello, FABRIC from node Node3
